# CasaOS Spotlight

In [12]:
# imports

from dotenv import load_dotenv
from IPython.display import display
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
import ipywidgets as widgets
import os
import pinecone

load_dotenv()

pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENV"])

index_name = "langchain-demo"
index = pinecone.Index(index_name=index_name)

embeddings = OpenAIEmbeddings()

docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True
)

status_text = widgets.Output()

summary_text = widgets.Output(
    style={'description_width': 'initial', 'font_size': '20px'}
)

search_result_text = widgets.Output(
    style={'description_width': 'initial', 'font_size': '20px'}
)


def demo(query):
    summary_text.clear_output(wait=False)
    search_result_text.clear_output(wait=False)

    with status_text:
        print("searching...")

    result = qa(
        {
            "query": query['new']
        }
    )

    with summary_text:
        if result["result"]:
            print(result["result"])
        else:
            print("No summary available.")

    with search_result_text:
        for doc in docsearch.similarity_search(query=query['new']):
            print(doc.metadata)


    status_text.clear_output(wait=False)


input_text = widgets.Text(
    continuous_update=False, 
    layout=widgets.Layout(width='62%'), placeholder='What do you want to know?',
    style={'description_width': 'initial', 'font_size': '16px'}
)

# Display widget
display(
    input_text, 
    widgets.Label(value="Summary", style={'font_size': '16px'}), 
    summary_text,
    widgets.Label(value="Search results", style={'font_size': '16px'}),
    search_result_text,
    status_text,
    )

input_text.observe(demo, names='value')

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 61}},
 'total_vector_count': 61}


Text(value='', continuous_update=False, layout=Layout(width='62%'), placeholder='What do you want to know?', s…

Label(value='Summary', style=LabelStyle(font_size='24px'))

Output()

Label(value='Search results', style=LabelStyle(font_size='24px'))

Output()

Output()